# MP Neuron

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import re

In [2]:
train = pd.read_csv(r"D:\git\PRACTICE\DATA\train.csv")
test = pd.read_csv(r"D:\git\PRACTICE\DATA\test.csv")

In [8]:
features = ["RAM", "Capacity", "Pixel Density", "Internal Memory"]
target = "Rating"

In [9]:
# Function to extract numbers from text
def extract_number(value):
    if pd.isnull(value):
        return np.nan
    value = str(value)

    match = re.search(r'\d+', value.replace(',', ''))
    if match:
        return float(match.group())
    return np.nan



In [10]:
for col in features:
    train[col] = train[col].apply(extract_number)
    test[col] = test[col].apply(extract_number)


In [11]:
for col in features:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())
    
  # use train median

In [12]:
train["target"] = (train["Rating"] > 4.5).astype(int)

In [13]:
X_train = train[features].values
y_train = train["target"].values
X_test = test[features].values

In [14]:
X_train = X_train / X_train.max(axis=0)
X_test = X_test / X_train.max(axis=0)

In [15]:
class MPNeuron:
    def __init__(self, input_size, threshold=0.5):
        self.weights = np.random.rand(input_size)
        self.threshold = threshold

    def predict(self, X):
        linear_output = np.dot(X, self.weights)
        return (linear_output > self.threshold).astype(int)

mp_neuron = MPNeuron(X_train.shape[1])

In [16]:
# Predictions
y_pred_train = mp_neuron.predict(X_train)
y_pred_test = mp_neuron.predict(X_test)

In [18]:
train_acc = np.mean(y_pred_train == y_train)
print("MP Neuron Train Accuracy:", train_acc)

MP Neuron Train Accuracy: 0.27887323943661974


In [19]:
print("MP Neuron Test Predictions (first 10 rows):", y_pred_test[:10])

MP Neuron Test Predictions (first 10 rows): [1 1 1 1 1 1 1 1 1 1]


In [ ]:
'''output = pd.DataFrame({
    "PhoneId": test["PhoneId"],
    "Class": y_pred_test
})
output.to_excel(r"D:\git\PRACTICE\DATA\OUTPUT\mp_neuron_test_predictions.xlsx", index=False)
print("Test predictions saved to mp_neuron_test_predictions.xlsx")'''

# final 
# Random weights
# Fixed threshold
# No learning from y_train

# PERCEPTRON

In [20]:
class Perceptron:
    def __init__(self, input_size, lr=0.01, epochs=1000):
        self.weights = np.zeros(input_size)
        self.bias = 0
        self.lr = lr
        self.epochs = epochs

    def activation(self, x):
        return np.where(x > 0, 1, 0)

    def fit(self, X, y):
        for _ in range(self.epochs):
            for xi, target in zip(X, y):
                linear_output = np.dot(xi, self.weights) + self.bias
                y_pred = self.activation(linear_output)
                error = target - y_pred
                # Update rule
                self.weights += self.lr * error * xi
                self.bias += self.lr * error

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self.activation(linear_output)
    

# Starts with zero weights

# Updates weights using training data

# Learns gradually → train accuracy improves

In [21]:
perceptron = Perceptron(X_train.shape[1], lr=0.01, epochs=1000)
perceptron.fit(X_train, y_train)

y_pred_train = perceptron.predict(X_train)
y_pred_test = perceptron.predict(X_test)

In [22]:
train_acc = np.mean(y_pred_train == y_train)
print("Perceptron Train Accuracy:", train_acc)


Perceptron Train Accuracy: 0.9577464788732394


In [ ]:
'''print("Perceptron Test Predictions (first 10 rows):", y_pred_test[:10])
output = pd.DataFrame({
    "PhoneId": test["PhoneId"],
    "Class": y_pred_test
})
output.to_excel(r"D:\git\PRACTICE\DATA\OUTPUT\perceptron_test_predictions.xlsx", index=False)
print("Test predictions saved to perceptron_test_predictions.xlsx")'''

Perceptron Test Predictions (first 10 rows): [1 1 1 1 1 1 1 1 1 1]
Test predictions saved to perceptron_test_predictions.xlsx


# SIGMOID

In [ ]:
features = [
    "RAM", "Capacity", "Pixel Density", "Internal Memory",
    "Height", "Width", "Screen Size", "Weight"
]

In [27]:
def extract_number(series):
    return series.astype(str).str.extract(r'([\d.]+)')[0].astype(float)

for col in features:
    train[col] = extract_number(train[col])
    test[col] = extract_number(test[col])
    
# Fill missing values with median
for col in features:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())

In [28]:
train["target"] = (train["Rating"] > 4.5).astype(int)

In [31]:
from sklearn.utils import resample

train_majority = train[train.target == 0]
train_minority = train[train.target == 1]

train_minority_upsampled = resample(
    train_minority,
    replace=True,
    n_samples=len(train_majority),
    random_state=42
)

train_balanced = pd.concat([train_majority, train_minority_upsampled])

In [32]:
X_train = train_balanced[features].values
y_train = train_balanced["target"].values
X_test = test[features].values

# Normalize
X_train = X_train / X_train.max(axis=0)
X_test = X_test / X_train.max(axis=0)

In [33]:
class SigmoidNeuron:
    def __init__(self, input_size, lr=0.1, epochs=1000):
        self.weights = np.random.rand(input_size)
        self.bias = 0
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        for _ in range(self.epochs):
            z = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(z)
            error = y - y_pred
            # Gradient descent update
            self.weights += self.lr * np.dot(X.T, error)
            self.bias += self.lr * error.mean()

    def predict_proba(self, X):
        z = np.dot(X, self.weights) + self.bias
        return self.sigmoid(z)

    def predict(self, X, threshold=0.5):
        probs = self.predict_proba(X)
        return (probs > threshold).astype(int)


In [34]:
sig_neuron = SigmoidNeuron(X_train.shape[1], lr=0.1, epochs=1000)
sig_neuron.fit(X_train, y_train)

In [35]:
y_pred_train = sig_neuron.predict(X_train, threshold=0.5)
train_acc = np.mean(y_pred_train == y_train)
print("Sigmoid Neuron Train Accuracy:", train_acc)

Sigmoid Neuron Train Accuracy: 0.644542772861357


# Sigmoid Image Data

In [ ]:

import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import shuffle

In [ ]:
def load_images_from_folder(folder_paths, label):
    X, y = [], []
    for folder in folder_paths:
        for file in os.listdir(folder):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(folder, file)
                img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
                arr = img_to_array(img)/255.0  # normalize
                X.append(arr)
                y.append(label)
    return X, y

In [ ]:
neg_folders = [r"D:\git\PRACTICE\DATA\level_1_train\level_1\background"]
pos_folders = [
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\en",
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\hi",
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\ta"
]

In [ ]:

def load_images_from_folder(folder_paths, label):

X_neg, y_neg = load_images_from_folder(neg_folders, 0)
X_pos, y_pos = load_images_from_folder(pos_folders, 1)

X = np.array(X_neg + X_pos)
y = np.array(y_neg + y_pos)

X = X.reshape((X.shape[0], -1))  

X, y = shuffle(X, y, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=X.shape[1]))  # single neuron

model
model.fit(X, y, epochs=100, batch_size=32, verbose=1)

In [ ]:
y_pred = (model.predict(X) > 0.5).astype(int).flatten()

acc = accuracy_score(y, y_pred)
prec = precision_score(y, y_pred, zero_division=0)
rec = recall_score(y, y_pred, zero_division=0)
f1 = f1_score(y, y_pred, zero_division=0)
cm = confusion_matrix(y, y_pred)

In [ ]:
df_pred = pd.DataFrame({
    "ImageId": np.arange(len(y_pred)),
    "Prediction": y_pred
})
df_pred.to_excel("predictions_sigmoid.xlsx", index=False)
print("Predictions saved to predictions_sigmoid.xlsx")

In [ ]:
print("\nSigmoid Neuron Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)